In [1]:
from awpy import Demo

import torch
from torch_geometric_temporal.signal import DynamicHeteroGraphTemporalSignal

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import time
import json
import sys
import os

pd.set_option('display.max_columns', 100)
pd.set_option('future.no_silent_downcasting', True)

pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../package'))

from CS2.graph import TabularGraphSnapshot, HeteroGraphSnapshot, TemporalHeteroGraphSnapshot
from CS2.token import Tokenizer
from CS2.preprocess import Dictionary, NormalizePosition, NormalizeTabularGraphSnapshot, ImputeTabularGraphSnapshot
from CS2.visualize import HeteroGraphVisualizer

### 1. Tabular Dataset Creation

#### 2024 Matches

Parse date: 2024. 09. 17.

In [6]:
# --------------------------------------------------
# PREPARATION
# --------------------------------------------------

MATCH_FOLDER_PATH = '../../data/matches-raw/cs2/2024/'
SAVE_PATH = '../../data/matches-processed/cs2/tabular/2024/'

# Get inferno matches
match_list = os.listdir(MATCH_FOLDER_PATH)
inferno_match_list = [file for file in match_list if 'inferno' in file and 
    '-p1' not in file and 
    '-p2' not in file and 
    '-p3' not in file and 
    '-p4' not in file and 
    '-p5' not in file
]

# Save complete match list and process time
parsed_matches_list = []
error_matches_list = []
process_time_list = []

# Match index
match_index = 0



# Read the files if they exist
if os.path.exists('completed_matches.txt'):
    with open('completed_matches.txt', 'r') as file:
        parsed_matches_list = [line.strip() for line in file]

if os.path.exists('error_matches.txt'):
    with open('error_matches.txt', 'r') as file:
        error_matches_list = [line.strip() for line in file]

if os.path.exists('process_times.txt'):
    with open('process_times.txt', 'r') as file:
        process_time_list = [line.strip() for line in file]

if os.path.exists('match_index.txt'):
    with open('match_index.txt', 'r') as file:
        match_index = int(file.read())


# Map nodes dataset
nodes = pd.read_csv('../../data/map_graph_model/de_inferno/nodes.csv')



# --------------------------------------------------
# PARSE MATCHES
# --------------------------------------------------

# Parse Inferno matches
for match in inferno_match_list:

    # If the match has already been parsed or is corrupted, skip it
    if match in parsed_matches_list or match in error_matches_list:
        continue

    # Start timer
    start_time = time.time()

    # Match path
    match_path = MATCH_FOLDER_PATH + match

    # Create tabular snapshot object
    tg = TabularGraphSnapshot()

    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('                                                        PARSING MATCH                                                        \n')
    print('Match: ' + match)

    try:
        # Create tabular snapshot database
        df, df_dict, active_infernos, active_smokes, active_he_smokes = tg.process_match(
            match_path=match_path,
            player_stats_data_path='../../data/player-stats/scraped-in-2024/2023/norm_player_stats_2023.csv',
            missing_player_stats_data_path='../../data/player-stats/missing_players_df_2023.csv',
            weapon_data_path='../../data/weapon_info/ammo_info.csv',

            ticks_per_second=4,
            numerical_match_id=100000 + match_index,
            num_permutations_per_round=1,
            build_dictionary=True,

            package='pandas'
        )

    except Exception as e:
        # Print info and save error matches
        print('Error occured while parsing the match. Skipping match.')

        error_matches_list.append(match)
        with open('error_matches.txt', 'w') as file:
            for item in error_matches_list:
                file.write(f"{item}\n")
                
        continue



    # Impute missing values
    its = ImputeTabularGraphSnapshot()
    df = its.impute(df)

    # Tokenize match
    tokenizer = Tokenizer()
    df = tokenizer.tokenize_match(df, 'de_inferno', nodes)

    # Save dataframes
    df.to_csv(SAVE_PATH + match + '_df.csv', index=False)
    df_dict.to_csv(SAVE_PATH + match + '_df_dict.csv', index=False)
    active_infernos.to_csv(SAVE_PATH + match + '_active_infernos.csv', index=False)
    active_smokes.to_csv(SAVE_PATH + match + '_active_smokes.csv', index=False)
    active_he_smokes.to_csv(SAVE_PATH + match + '_active_he_smokes.csv', index=False)

    # Post-process
    match_index += 1
    parsed_matches_list.append(match)



    # --------------------------------------------------
    # LOG AND SAVE TIME
    # --------------------------------------------------
    
    # Time
    end_time = time.time()
    process_time = end_time - start_time
    process_time_list.append(process_time)

    # Write results to file
    with open('completed_matches.txt', 'w') as file:
        for item in parsed_matches_list:
            file.write(f"{item}\n")
    
    with open('process_times.txt', 'w') as file:
        for item in process_time_list:
            file.write(f"{item}\n")
    
    with open('match_index.txt', 'w') as file:
        file.write(str(match_index))

    print('Parse completed. Duration: ' + str(process_time) + ' seconds.')

-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-dallas-2024-virtuspro-vs-big-bo3-iNJygOnxyHMLcZ0aB_CRymvirtus-pro-vs-big-m1-inferno.dem
Parse completed. Duration: 95.31853127479553 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-katowice-2024-eternal-fire-vs-faze-bo3-NSvOwra3ZJLAlQou2jhAKveternal-fire-vs-faze-m1-inferno.dem
Parse completed. Duration: 199.3240532875061 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                   

#### 2023 Matches

Parse date: 2024. 09. 18.

In [41]:
# --------------------------------------------------
# PREPARATION
# --------------------------------------------------

MATCH_FOLDER_PATH = '../../data/matches-raw/cs2/2023/'
SAVE_PATH = '../../data/matches-processed/cs2/tabular/2023/'

# Get inferno matches
match_list = os.listdir(MATCH_FOLDER_PATH)
inferno_match_list = [file for file in match_list if 'inferno' in file and 
    '-p1' not in file and 
    '-p2' not in file and 
    '-p3' not in file and 
    '-p4' not in file and 
    '-p5' not in file
]

# Save complete match list and process time
parsed_matches_list = []
error_matches_list = []
process_time_list = []

# Match index
match_index = 0



# Read the files if they exist
if os.path.exists('completed_matches.txt'):
    with open('completed_matches.txt', 'r') as file:
        parsed_matches_list = [line.strip() for line in file]

if os.path.exists('error_matches.txt'):
    with open('error_matches.txt', 'r') as file:
        error_matches_list = [line.strip() for line in file]

if os.path.exists('process_times.txt'):
    with open('process_times.txt', 'r') as file:
        process_time_list = [line.strip() for line in file]

if os.path.exists('match_index.txt'):
    with open('match_index.txt', 'r') as file:
        match_index = int(file.read())


# Map nodes dataset
nodes = pd.read_csv('../../data/map_graph_model/de_inferno/nodes.csv')



# --------------------------------------------------
# PARSE MATCHES
# --------------------------------------------------

# Parse Inferno matches
for match in inferno_match_list:

    # If the match has already been parsed or is corrupted, skip it
    if match in parsed_matches_list or match in error_matches_list:
        continue

    # Start timer
    start_time = time.time()

    # Match path
    match_path = MATCH_FOLDER_PATH + match

    # Create tabular snapshot object
    tg = TabularGraphSnapshot()

    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('                                                        PARSING MATCH                                                        \n')
    print('Match: ' + match)

    try:
        # Create tabular snapshot database
        df, df_dict, active_infernos, active_smokes, active_he_smokes = tg.process_match(
            match_path=match_path,
            player_stats_data_path='../../data/player-stats/scraped-in-2024/2022/norm_player_stats_2022.csv',
            missing_player_stats_data_path='../../data/player-stats/missing_players_df_2022.csv',
            weapon_data_path='../../data/weapon_info/ammo_info.csv',

            ticks_per_second=4,
            numerical_match_id=100000 + match_index,
            num_permutations_per_round=1,
            build_dictionary=True,

            package='pandas'
        )

        # Impute missing values
        its = ImputeTabularGraphSnapshot()
        df = its.impute(df)

        # Tokenize match
        tokenizer = Tokenizer()
        df = tokenizer.tokenize_match(df, 'de_inferno', nodes)

        # Save dataframes
        df.to_csv(SAVE_PATH + match + '_df.csv', index=False)
        df_dict.to_csv(SAVE_PATH + match + '_df_dict.csv', index=False)
        active_infernos.to_csv(SAVE_PATH + match + '_active_infernos.csv', index=False)
        active_smokes.to_csv(SAVE_PATH + match + '_active_smokes.csv', index=False)
        active_he_smokes.to_csv(SAVE_PATH + match + '_active_he_smokes.csv', index=False)

    except Exception as e:
        # Print info and save error matches
        print('Error occured while parsing the match. Skipping match.')

        error_matches_list.append(match)
        with open('error_matches.txt', 'w') as file:
            for item in error_matches_list:
                file.write(f"{item}\n")
                
        continue


    # Post-process
    match_index += 1
    parsed_matches_list.append(match)



    # --------------------------------------------------
    # LOG AND SAVE TIME
    # --------------------------------------------------
    
    # Time
    end_time = time.time()
    process_time = end_time - start_time
    process_time_list.append(process_time)

    # Write results to file
    with open('completed_matches.txt', 'w') as file:
        for item in parsed_matches_list:
            file.write(f"{item}\n")
    
    with open('process_times.txt', 'w') as file:
        for item in process_time_list:
            file.write(f"{item}\n")
    
    with open('match_index.txt', 'w') as file:
        file.write(str(match_index))

    print('Parse completed. Duration: ' + str(process_time) + ' seconds.')

-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-sydney-2023-mouz-vs-vertex-inferno-wyV4hHh3mDqNZhY8zBC_ttmouz-vs-vertex-inferno.dem


2024-09-17 20:54:00.247 | WARNING  | awpy.parsers.events:parse_bomb:279 - bomb_defused not found in events.


Error occured while parsing the match. Skipping match.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-sydney-2023-natus-vincere-vs-mouz-bo3-kmz24m_e2rc4e07yRHJsFcnatus-vincere-vs-mouz-m2-inferno.dem


2024-09-17 20:55:20.532 | WARNING  | awpy.parsers.events:parse_bomb:309 - bomb_exploded not found in events.


Error occured while parsing the match. Skipping match.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-sydney-2023-vitality-vs-faze-bo3-KYy8n7ELEDs3OUXK1CHiYpvitality-vs-faze-m2-inferno.dem
Error occured while parsing the match. Skipping match.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _roobet-cup-2023-fnatic-vs-ninjas-in-pyjamas-bo3-vvaeYNK3nyhj-S-LM5M0XFfnatic-vs-ninjas-in-pyjamas-m1-inferno.dem
Parse completed. Duration: 136.1281979084015 seconds.
--------------------------------------------------------------------------------------------------------------

2024-09-17 21:07:33.113 | WARNING  | awpy.parsers.events:parse_bomb:279 - bomb_defused not found in events.


Parse completed. Duration: 81.6461410522461 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _thunderpick-world-championship-2023-fnatic-vs-ninjas-in-pyjamas-bo3-6iVaParaf7r9nrM0X_nIUKfnatic-vs-ninjas-in-pyjamas-m2-inferno.dem
Parse completed. Duration: 195.55537915229797 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _thunderpick-world-championship-2023-fnatic-vs-virtuspro-bo3-TIXlP91jgLe7XdTNek2melfnatic-vs-virtus-pro-m2-inferno.dem
Parse completed. Duration: 288.2054376602173 seconds.
-----------------------------------------------------------------

### 2. Heterogeneous Graph Dataset Creation

#### 2.1 Create Dictionary

In [2]:
# File paths
MATCH_FOLDER_PATH_2023 = '../../data/matches-processed/cs2/tabular/2023/'
MATCH_FOLDER_PATH_2024 = '../../data/matches-processed/cs2/tabular/2024/'

# Get files in the folders
file_list_2023 = os.listdir(MATCH_FOLDER_PATH_2023)
file_list_2024 = os.listdir(MATCH_FOLDER_PATH_2024)



# --------------------------------------------------
# Build Dictionary
# --------------------------------------------------

# Get dictionary files
df_dict_list_2023 = [file for file in file_list_2023 if file.endswith('_df_dict.csv')]
df_dict_list_2024 = [file for file in file_list_2024 if file.endswith('_df_dict.csv')]

# Build dictionaries
dict_2023 = Dictionary().build_dictionary(MATCH_FOLDER_PATH_2023, df_dict_list_2023)
dict_2024 = Dictionary().build_dictionary(MATCH_FOLDER_PATH_2024, df_dict_list_2024)

# Merge dictionaries
dictionary = Dictionary().merge_dictionaries([dict_2023, dict_2024])

# Save dictionary
dictionary.to_csv('dictionary_2023-2024.09.csv', index=False)

#### 2.2 Create Heterogeneous Graph Snahpots

In [3]:
# Get CONFIG parameters
inferno_pos_norm_config = '../../config/map_normalization/inferno.json'
molotov_radius_config = '../../config/nade_radius/molotov_norm.json'
smoke_radius_config = '../../config/nade_radius/smoke_norm.json'

with open(inferno_pos_norm_config, 'r') as f:
    CONFIG_INF_POS_NORM = json.load(f)
with open(molotov_radius_config, 'r') as f:
    CONFIG_MOLOTOV_RADIUS = json.load(f)
with open(smoke_radius_config, 'r') as f:
    CONFIG_SMOKE_RADIUS = json.load(f)

##### 2023

In [6]:
# Graph save path
GRAPH_SAVE_PATH = '../../data/matches-processed/cs2/hetero-graph/'

# Get match list
df_match_list_2023 = [file for file in file_list_2023 if file.endswith('_df.csv')]

# Nodes and edges dataframes
nodes_to_use = pd.read_csv('../../data/map_graph_model/de_inferno/nodes_norm.csv')
edges = pd.read_csv('../../data/map_graph_model/de_inferno/edges.csv')

# Save complete match list and process time
parsed_matches_list = []
error_matches_list = []
process_time_list = []

# Read the files if they exist
if os.path.exists('completed_matches.txt'):
    with open('completed_matches.txt', 'r') as file:
        parsed_matches_list = [line.strip() for line in file]

if os.path.exists('error_matches.txt'):
    with open('error_matches.txt', 'r') as file:
        error_matches_list = [line.strip() for line in file]

if os.path.exists('process_times.txt'):
    with open('process_times.txt', 'r') as file:
        process_time_list = [line.strip() for line in file]


# Iterate over the matches
for match in df_match_list_2023:

    # Start timer
    start_time = time.time()

    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('                                                        PARSING MATCH                                                        \n')
    print('Match: ' + match)

    try:
        # Read dataframes
        df = pd.read_csv(MATCH_FOLDER_PATH_2023 + match)
        active_infernos = pd.read_csv(MATCH_FOLDER_PATH_2023 + match.replace('_df.csv', '_active_infernos.csv'))
        active_smokes = pd.read_csv(MATCH_FOLDER_PATH_2023 + match.replace('_df.csv', '_active_smokes.csv'))
        try:
            active_he_smokes = pd.read_csv(MATCH_FOLDER_PATH_2023 + match.replace('_df.csv', '_active_he_smokes.csv'))
        except:
            active_he_smokes = pd.DataFrame(columns=['tick', 'round', 'X', 'Y', 'Z'])
            print('No HE thrown into smokes in the match. Creating empty dataframe.')

        # Normalize active nade dataframes
        np = NormalizePosition()
        active_infernos = np.normalize(active_infernos, CONFIG_INF_POS_NORM)
        active_smokes = np.normalize(active_smokes, CONFIG_INF_POS_NORM)
        active_he_smokes = np.normalize(active_he_smokes, CONFIG_INF_POS_NORM)

        # Normalize tabular snapshot
        nts = NormalizeTabularGraphSnapshot()
        df = nts.noramlize(df, dictionary, CONFIG_INF_POS_NORM)

        # Graph snapshots
        hg = HeteroGraphSnapshot()
        graphs = hg.process_snapshots(df, nodes_to_use, edges, active_infernos, active_smokes, active_he_smokes, CONFIG_MOLOTOV_RADIUS, CONFIG_SMOKE_RADIUS)

        # Save graph snapshots
        torch.save(graphs, GRAPH_SAVE_PATH + str(df['NUMERICAL_MATCH_ID'].iloc[0]) + '.pt')

        # Save post-processed match
        parsed_matches_list.append(match)
    
        # Time
        end_time = time.time()
        process_time = end_time - start_time
        process_time_list.append(process_time)

        # Write results to file
        with open('completed_matches.txt', 'w') as file:
            for item in parsed_matches_list:
                file.write(f"{item}\n")
        
        with open('process_times.txt', 'w') as file:
            for item in process_time_list:
                file.write(f"{item}\n")

        print('Parse completed. Duration: ' + str(process_time) + ' seconds.')


    except Exception as e:
        
        # Print info and save error matches
        print('Error occured while parsing, skipping match. Error message:')
        print(e)

        error_matches_list.append(match)
        with open('error_matches.txt', 'w') as file:
            for item in error_matches_list:
                file.write(f"{item}\n")
                
        continue
    

-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _blast-premier-fall-final-2023-cloud9-vs-heroic-bo3-FZhx5bice3pWss26wFHIIYcloud9-vs-heroic-m2-inferno.dem_df.csv
Parse completed. Duration: 77.31664419174194 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _blast-premier-fall-final-2023-cloud9-vs-natus-vincere-bo3-vxKYiMWUPLiYn9oyBq6Njdcloud9-vs-natus-vincere-m3-inferno.dem_df.csv
Parse completed. Duration: 86.16453409194946 seconds.
-----------------------------------------------------------------------------------------------------------------------------
      

C:\Users\szmid\AppData\Local\Temp\ipykernel_3928\2388612964.py:42: DtypeWarning: Columns (1677,1678) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MATCH_FOLDER_PATH_2023 + match)


Parse completed. Duration: 81.00898814201355 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _cs-asia-championships-2023-mouz-vs-wings-up-inferno-rBbDAQbfvHTcx042LMx-mOmouz-vs-wings-up-inferno.dem_df.csv


C:\Users\szmid\AppData\Local\Temp\ipykernel_3928\2388612964.py:42: DtypeWarning: Columns (1677,1678) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MATCH_FOLDER_PATH_2023 + match)


Parse completed. Duration: 55.29296350479126 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-sydney-2023-gamerlegion-vs-faze-bo3-8GCYTI5P5NZCLcg9Xq9-bagamerlegion-vs-faze-m2-inferno.dem_df.csv
No HE thrown into smokes in the match. Creating empty dataframe.
Parse completed. Duration: 83.00204491615295 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _iem-sydney-2023-mouz-vs-vertex-inferno-wyV4hHh3mDqNZhY8zBC_ttmouz-vs-vertex-inferno.dem_df.csv
No HE thrown into smokes in the match. Creating empty dataframe.
Parse completed. Duration: 50.85057687759

##### 2024

In [5]:
# Graph save path
GRAPH_SAVE_PATH = '../../data/matches-processed/cs2/hetero-graph/'

# Get match list
df_match_list_2024 = [file for file in file_list_2024 if file.endswith('_df.csv')]

# Nodes and edges dataframes
nodes_to_use = pd.read_csv('../../data/map_graph_model/de_inferno/nodes_norm.csv')
edges = pd.read_csv('../../data/map_graph_model/de_inferno/edges.csv')

# Save complete match list and process time
parsed_matches_list = []
error_matches_list = []
process_time_list = []

# Read the files if they exist
if os.path.exists('completed_matches.txt'):
    with open('completed_matches.txt', 'r') as file:
        parsed_matches_list = [line.strip() for line in file]

if os.path.exists('error_matches.txt'):
    with open('error_matches.txt', 'r') as file:
        error_matches_list = [line.strip() for line in file]

if os.path.exists('process_times.txt'):
    with open('process_times.txt', 'r') as file:
        process_time_list = [line.strip() for line in file]


# Iterate over the matches
for match in df_match_list_2024:

    # Start timer
    start_time = time.time()

    print('-----------------------------------------------------------------------------------------------------------------------------')
    print('                                                        PARSING MATCH                                                        \n')
    print('Match: ' + match)

    try:
        # Read dataframes
        df = pd.read_csv(MATCH_FOLDER_PATH_2024 + match)
        active_infernos = pd.read_csv(MATCH_FOLDER_PATH_2024 + match.replace('_df.csv', '_active_infernos.csv'))
        active_smokes = pd.read_csv(MATCH_FOLDER_PATH_2024 + match.replace('_df.csv', '_active_smokes.csv'))
        try:
            active_he_smokes = pd.read_csv(MATCH_FOLDER_PATH_2024 + match.replace('_df.csv', '_active_he_smokes.csv'))
        except:
            active_he_smokes = pd.DataFrame(columns=['tick', 'round', 'X', 'Y', 'Z'])
            print('No HE thrown into smokes in the match. Creating empty dataframe.')

        # Normalize active nade dataframes
        np = NormalizePosition()
        active_infernos = np.normalize(active_infernos, CONFIG_INF_POS_NORM)
        active_smokes = np.normalize(active_smokes, CONFIG_INF_POS_NORM)
        active_he_smokes = np.normalize(active_he_smokes, CONFIG_INF_POS_NORM)

        # Normalize tabular snapshot
        nts = NormalizeTabularGraphSnapshot()
        df = nts.noramlize(df, dictionary, CONFIG_INF_POS_NORM)

        # Graph snapshots
        hg = HeteroGraphSnapshot()
        graphs = hg.process_snapshots(df, nodes_to_use, edges, active_infernos, active_smokes, active_he_smokes, CONFIG_MOLOTOV_RADIUS, CONFIG_SMOKE_RADIUS)

        # Save graph snapshots
        torch.save(graphs, GRAPH_SAVE_PATH + str(df['NUMERICAL_MATCH_ID'].iloc[0]) + '.pt')

        # Save post-processed match
        parsed_matches_list.append(match)
    
        # Time
        end_time = time.time()
        process_time = end_time - start_time
        process_time_list.append(process_time)

        # Write results to file
        with open('completed_matches.txt', 'w') as file:
            for item in parsed_matches_list:
                file.write(f"{item}\n")
        
        with open('process_times.txt', 'w') as file:
            for item in process_time_list:
                file.write(f"{item}\n")

        print('Parse completed. Duration: ' + str(process_time) + ' seconds.')


    except Exception as e:
        
        # Print info and save error matches
        print('Error occured while parsing, skipping match. Error message:')
        print(e)

        error_matches_list.append(match)
        with open('error_matches.txt', 'w') as file:
            for item in error_matches_list:
                file.write(f"{item}\n")
                
        continue
    

-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _betboom-dacha-belgrade-season-2-eternal-fire-vs-pain-bo3-EsO1ghXhDey0Ii85scyL6seternal-fire-vs-pain-m1-inferno.dem_df.csv
Parse completed. Duration: 106.62941002845764 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _betboom-dacha-belgrade-season-2-mouz-vs-eternal-fire-bo3-5Hd0ELRRmki7NgbHcMS0LRmouz-vs-eternal-fire-m1-inferno.dem_df.csv
Parse completed. Duration: 90.96409463882446 seconds.
-----------------------------------------------------------------------------------------------------------------------------

C:\Users\szmid\AppData\Local\Temp\ipykernel_1500\937563431.py:42: DtypeWarning: Columns (0,820) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MATCH_FOLDER_PATH_2024 + match)


Parse completed. Duration: 231.8471896648407 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _pgl-cs2-major-copenhagen-2024-virtuspro-vs-g2-bo3-_HljkSgUssxnv7gY6zueWRvirtus-pro-vs-g2-m2-inferno.dem_df.csv
Parse completed. Duration: 149.32663083076477 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _pgl-cs2-major-copenhagen-2024-virtuspro-vs-imperial-inferno-khjhOWhI70GAS-g9WWfGzcvirtus-pro-vs-imperial-inferno.dem_df.csv
Parse completed. Duration: 119.89045286178589 seconds.
-------------------------------------------------------------------------------

C:\Users\szmid\AppData\Local\Temp\ipykernel_1500\937563431.py:42: DtypeWarning: Columns (0,820) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MATCH_FOLDER_PATH_2024 + match)


Parse completed. Duration: 84.90058851242065 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _yalla-compass-2024-complexity-vs-big-inferno-0NOZtg3Sw8z-wQPgDTN6_8complexity-vs-big-inferno.dem_df.csv
Parse completed. Duration: 74.80462288856506 seconds.
-----------------------------------------------------------------------------------------------------------------------------
                                                        PARSING MATCH                                                        

Match: _yalla-compass-2024-complexity-vs-furia-inferno-nNF-QzYEPXKM32ZGDbj_Xqcomplexity-vs-furia-inferno.dem_df.csv
Parse completed. Duration: 99.80917072296143 seconds.
--------------------------------------------------------------------------------------------------------